In [ ]:
import faiss
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
from PIL import Image
from deepface import DeepFace
from scipy.spatial.distance import cosine
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model

In [ ]:
target_size = (224, 224)
batch_size = 8
images_path = glob.glob("../my_images/*")

In [ ]:
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(target_size[0], target_size[1], 3))

feature_extractor = Model(inputs=base_model.input, outputs=base_model.output)

def extract_feature(img_path):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    features = feature_extractor.predict(img_array)
    features = features.flatten()

    return features

In [ ]:
image_features = []
for img_path in images_path:
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    extracted_vector = extract_feature(img_path).flatten()
    image_features.append(extracted_vector)

image_features = np.array(image_features)

In [ ]:
index = faiss.IndexFlatL2(image_features.shape[1])
index.add(image_features)

def find_closest(img_path):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    extracted_vector = feature_extractor.predict(img_array).flatten()
    extracted_vector = np.array([extracted_vector])
    _, result_search = index.search(extracted_vector, 1)

    return images_path[result_search[0][0]]

In [ ]:
def is_it_me(img_path):
    extracted_vector = extract_feature(img_path)
    for my_img_vector in image_features:
        similarity = 1 - cosine(my_img_vector, extracted_vector)
        print(similarity)
        if similarity > 0.7:
            return True
    return False
    
print("This is you Abolfazl!") if is_it_me("../test_images/test.jpg") else print("This is not you!")

In [ ]:
embeddings = []
for my_img in images_path:
    try:
        embedding = DeepFace.represent(img_path=my_img, model_name="Facenet")
    except:
        print("Face has not detected!")
    embeddings.append(embedding[0]["embedding"])
embeddings

In [ ]:
def is_it_me(img):
    embedding = DeepFace.represent("../test_images/test.jpg", model_name="Facenet")
    embedding_vector = embedding[0]["embedding"]
    fa = embedding[0]["facial_area"]
    x, y, w, h = fa["x"], fa["y"], fa["w"], fa["h"]
    for em in embeddings:
        similarity = 1 - cosine(em, embedding_vector)
        if similarity > 0.7:
            return embedding[0], True, (x, y, w, h)
    return None, False, None

In [ ]:
cam = cv2.VideoCapture(0)
cv2.namedWindow("Abolfazl Detector")

while True:
    ret, frame = cam.read()
    if not ret:
        print("Failed to grab frame")
        break
    
    cv2.imshow("Abolfazl Detector", frame)
    
    if cv2.waitKey(1) & 0xFF == 13:
        embedding, result, face_info = is_it_me(frame)

        if result:
            x, y, w, h = face_info

            text = "This is you, Abolfazl!" if result else print("This is not you.")
            color = (0, 255, 0) if result else (0, 0, 255)
            
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)

            cv2.putText(frame, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 
                                0.8, color, 2, cv2.LINE_AA)
            
            cv2.imshow("Abolfazl Detector", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cam.release()
cv2.destroyAllWindows()